# Redes Neurais Artificiais


Ilustra o funcionamento do algoritmo de redes neurais.

Este notebook foi desenvolvido para o ambiente GOOGLE COLAB ([colab.research.google.com](https://colab.research.google.com)).

Prof. Hugo de Paula

-------------------------------------------------------------------------------

### Base de dados: Sonar, Mines vs. Rocks

https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+%28Sonar,+Mines+vs.+Rocks%29

208 instâncias

60 atributos

2 classes (rocha, mina)



In [0]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np

np.set_printoptions(precision=2)



### Carga dos dados


In [6]:
#from google.colab import files
#uploaded = files.upload()
#sonar = pd.read_excel('sonar.xlsx', sheet_name=0) 
from google.colab import drive
drive.mount('/content/drive')
sonar = pd.read_excel('/content/drive/My Drive/PUC/ML/Datasets/sonar.xlsx', sheet_name=0)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



### Transformação de dados

A classe é convertida para labels únicos sequenciais.

<code>
 le = preprocessing.LabelEncoder()
  
 le.fit(dados)
</code>


### Particionamento da base

<code>train_test_split(X, y) -- particiona a base de dados original em bases de treinamento e teste.</code>

No código a seguir, são utilizados 10% para teste e 90% para treinamento.


In [0]:
X = sonar.iloc[:,0:(sonar.shape[1] - 1)]

le = LabelEncoder()
y = le.fit_transform(sonar.iloc[:,(sonar.shape[1] - 1)])

class_names = le.classes_


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)



### MLP com uma camada oculta

O bloco a seguir testa a rede neural com um camada oculta com 100 neurônios. 

São totalizados 6.100 pesos diferentes que precisarão ser ajustados na fase de treinamento.

O parâmetro solver='lbfgs' foi escolhido por ser mais adequado para treinamento com bases pequenas (menores que alguns milhares de registros).

In [8]:
# Rede Perceptron Multicamadas (MLP):  Configuração default otimizando a função log-loss
# uma camada oculta com 100 neurônios.

mlp = MLPClassifier(solver='lbfgs', random_state=0)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)


print("Camadas da rede: {}".format(mlp.n_layers_))
print("Neurônios na camada oculta: {}".format(mlp.hidden_layer_sizes))
print("Neurônios na camada de saída: {}".format(mlp.n_outputs_))
print("Pesos na camada de entrada: {}".format(mlp.coefs_[0].shape))
print("Pesos na camada oculta: {}".format(mlp.coefs_[1].shape))

print("Acurácia da base de treinamento: {:.2f}".format(mlp.score(X_train, y_train)))
print("Acurácia da base de teste: {:.2f}".format(mlp.score(X_test, y_test)))


print(classification_report(y_test, y_pred, target_names=class_names))

# Calcula a matriz de confusão
cnf_matrix = confusion_matrix(y_test, y_pred)
print(cnf_matrix)

Camadas da rede: 3
Neurônios na camada oculta: (100,)
Neurônios na camada de saída: 1
Pesos na camada de entrada: (60, 100)
Pesos na camada oculta: (100, 1)
Acurácia da base de treinamento: 1.00
Acurácia da base de teste: 0.86
              precision    recall  f1-score   support

        Mina       0.78      0.88      0.82         8
       Rocha       0.92      0.85      0.88        13

    accuracy                           0.86        21
   macro avg       0.85      0.86      0.85        21
weighted avg       0.86      0.86      0.86        21

[[ 7  1]
 [ 2 11]]


### MLP com duas camadas ocultas

O bloco a seguir testa a rede neural com duas camadas ocultas. 

A primeira camada possui 100 neurônios, enquanto a segunda camada possui 60 neurônios. 

São totalizados 12.100 pesos diferentes que precisarão ser ajustados na fase de treinamento.

Com essa rede será possível observar que aumentar arbitrariamente a dimensão da sua rede neural não garante um aumento arbitrário da performance do modelo.

In [9]:

mlp = MLPClassifier(solver='lbfgs', random_state=0, hidden_layer_sizes=[100, 60])
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)


print("Camadas da rede: {}".format(mlp.n_layers_))
print("Neurônios na camada oculta: {}".format(mlp.hidden_layer_sizes))
print("Neurônios na camada de saída: {}".format(mlp.n_outputs_))
print("Pesos na camada de entrada: {}".format(mlp.coefs_[0].shape))
print("Pesos na camada oculta: {}".format(mlp.coefs_[1].shape))

print("Acurácia da base de treinamento: {:.2f}".format(mlp.score(X_train, y_train)))
print("Acurácia da base de teste: {:.2f}".format(mlp.score(X_test, y_test)))

print(classification_report(y_test, y_pred, target_names=class_names))

cnf_matrix = confusion_matrix(y_test, y_pred)
print(cnf_matrix)


Camadas da rede: 4
Neurônios na camada oculta: [100, 60]
Neurônios na camada de saída: 1
Pesos na camada de entrada: (60, 100)
Pesos na camada oculta: (100, 60)
Acurácia da base de treinamento: 1.00
Acurácia da base de teste: 0.86
              precision    recall  f1-score   support

        Mina       0.86      0.75      0.80         8
       Rocha       0.86      0.92      0.89        13

    accuracy                           0.86        21
   macro avg       0.86      0.84      0.84        21
weighted avg       0.86      0.86      0.86        21

[[ 6  2]
 [ 1 12]]


### *Overfitting* por excesso de neurônios

A forma mais eficiente para se determinar o número de neurônios na camada oculta é por busca sistemática. Um artigo interessante que ilustra diversas heurísticas para resolver o problema pode ser visto em 

 D. Stathakis (2009) *How many hidden layers and nodes?*, **International Journal of Remote Sensing**, 30:8, 2133-2147, DOI: 10.1080/01431160802549278 
 
 Entretanto, um ponto de partida inicial muito utilizado corresponde ao:
 
 (num_entradas + num_saídas) / 2.
 
 Note que a rede manteve a mesma performance das topologias anteriores.

In [10]:
mlp = MLPClassifier(solver='lbfgs', random_state=0, hidden_layer_sizes=[31])
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)


print("Camadas da rede: {}".format(mlp.n_layers_))
print("Neurônios na camada oculta: {}".format(mlp.hidden_layer_sizes))
print("Neurônios na camada de saída: {}".format(mlp.n_outputs_))
print("Pesos na camada de entrada: {}".format(mlp.coefs_[0].shape))
print("Pesos na camada oculta: {}".format(mlp.coefs_[1].shape))

print("Acurácia da base de treinamento: {:.2f}".format(mlp.score(X_train, y_train)))
print("Acurácia da base de teste: {:.2f}".format(mlp.score(X_test, y_test)))

print(classification_report(y_test, y_pred, target_names=class_names))

cnf_matrix = confusion_matrix(y_test, y_pred)
print(cnf_matrix)


Camadas da rede: 3
Neurônios na camada oculta: [31]
Neurônios na camada de saída: 1
Pesos na camada de entrada: (60, 31)
Pesos na camada oculta: (31, 1)
Acurácia da base de treinamento: 1.00
Acurácia da base de teste: 0.86
              precision    recall  f1-score   support

        Mina       0.73      1.00      0.84         8
       Rocha       1.00      0.77      0.87        13

    accuracy                           0.86        21
   macro avg       0.86      0.88      0.86        21
weighted avg       0.90      0.86      0.86        21

[[ 8  0]
 [ 3 10]]


### Ajustamento dos dados

As redes neurais a seguir irão testar a hipótese de que uma MLP é robusta quanto a dados não normalizados. 

Os dados serão padronizados pelo Z-score.

A normalização dos dados não irá acarretar em nenhume melhoria da rede.

In [14]:
# Calcula a média e o desvio padrão de cada atributo da base de treinamento
mean_on_train = X_train.mean(axis=0)
std_on_train = X_train.std(axis=0)

# Normaliza os atributos pela norma Z = (X - média) / desvio padrão
# afterwards, mean=0 and std=1
X_train_scaled = (X_train - mean_on_train) / std_on_train
# usa a esma transformação nos dados de teste
X_test_scaled = (X_test - mean_on_train) / std_on_train


# A rede atinge o número máximo de iterações, mas não converge.
mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=[31], random_state=0)
mlp.fit(X_train_scaled, y_train)
print("Acurácia da base de treinamento: {:.2f}".format(mlp.score(X_train_scaled, y_train)))
print("Acurácia da base de teste: {:.2f}".format(mlp.score(X_test_scaled, y_test)))

# Vamos aumentar o número máximo de iterações
mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=[31], max_iter=1000, random_state=0)
mlp.fit(X_train_scaled, y_train)
print("Acurácia da base de treinamento: {:.2f}".format(mlp.score(X_train_scaled, y_train)))
print("Acurácia da base de teste: {:.2f}".format(mlp.score(X_test_scaled, y_test)))


Acurácia da base de treinamento: 1.00
Acurácia da base de teste: 0.86
Acurácia da base de treinamento: 1.00
Acurácia da base de teste: 0.86
